# Reporting dashboard
## A prototype

In [6]:
from datetime import date
from IPython.display import display, HTML

import src.compute.changelogs as changelogs
from src.compute.utils import Interval
from src.db.utils import SnowflakeWrapper

conn = SnowflakeWrapper.create_snowflake_connection()
sw = SnowflakeWrapper(conn)

First, we need to define an interval in which we will operate:

In [7]:
active_interval: Interval = Interval(
    fromDate=date(2019, 10, 1), # year, month, day
    toDate=date(2020, 1, 1)
)

Number of actions on the given interval

In [8]:
active_cards: int = changelogs.unique_active_cards(sw, interval=active_interval)
print(f"Cards active in the interval {active_interval}: {active_cards}")

Cards active in the interval ('2019-10-01', '2020-01-01'): 2969


In [9]:
print(f"Most frequent transitions in the interval:")
most_frequent = changelogs.transition_frequency(sw, interval=active_interval, limit=10)
display(most_frequent)

Most frequent transitions in the interval:


,FromStatus,ToStatus,TotalTransitions
0,Ready for dev,Development,480
1,Development,Code review,450
2,Code review,Testing,430
3,In Backlog,Cancelled,398
4,Testing,Master,380
5,Master,Live,364
6,Code review,Needs CR fixes,277
7,Pending Review,Review,275
8,Needs CR fixes,Code review,267
9,Review,Ready for dev,256


In [10]:
print(f"Least frequent transitions in the interval:")
least_frequent = changelogs.transition_frequency(sw, interval=active_interval, limit=10, order="ASC")
display(least_frequent)

Least frequent transitions in the interval:


,FromStatus,ToStatus,TotalTransitions
0,Needs SR feedback,Cancelled,1
1,To-Do,Spec design,1
2,Cancelled,To-Do,1
3,Needs fixes,Done,1
4,On Hold,Development Backlog,1
5,To Do,Pending Review,1
6,Needs QA fixes,Testing On Hold,1
7,Work in Progress,To-Do,1
8,Backlog,Review,1
9,Ready for dev,On Hold,1
